# Sample level RNA-seq quality control

This pipeline implements RNA-seq QC that identifies and removes genes and samples from the TPM matrix.

## Methods overview

### GTEx V8 QC procedures

From https://gtexportal.org/home/documentationPage#staticTextAnalysisMethods : 


1. RNA-seq expression outliers were identified and excluded using a multidimensional extension of the statistic described in (Wright et al., Nat. Genet. 2014 ). Briefly, for each tissue, read counts from each sample were normalized using size factors calculated with DESeq2 and log-transformed with an offset of 1; genes with a log-transformed value >1 in >10% of samples were selected, and the resulting read counts were centered and unit-normalized. The resulting matrix was then hierarchically clustered (based on average and cosine distance), and a chi2 p-value was calculated based on Mahalanobis distance. Clusters with ≥60% samples with Bonferroni-corrected p-values <0.05 were marked as outliers, and their samples were excluded.
2. Samples with <10 million mapped reads were removed.
3. Genes were selected based on expression thresholds of >0.1 TPM in at least 20% of samples and ≥6 reads in at least 20% of samples. 

### Our implementation

For #1 and #2 above because the GTEx source code is not available, we adopted [some R scripts developed in this paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-021-04307-0) to perform gene and sample QC using TPM data, which we implements GTEx recommendations. #3 above is implemented in the gene expression normalization module.


## Three statistics to identify outliers

### RLE

RLE(Relative Log Expression)is assume that most expressions in a sample should be near a mean value, only a few genes have differential expression, which means higher or lower than other genes. To calculate it, for each gene *i* , calculate it median in *N* sample as *Medi* , for each sample *j* , and expression value *eij*, count the difference between *eij* and *Medi* : *deij = eij-Medi* , than boxplot each sample base on *deij* and sort by IQR(interquartile range), the sample with lager IQR is more likely to be an outlier. 

### Hierarchical clustering

We use(1 - spearman's correlation) as distances between samples and assume samples should be homogeneous, so all samples should have short distances between others and clustered homogeneously. So samples far away others may be outliers. So we first use distances which is 1-spearmen correlation to hierarchically clustered, than use top 100 gene sort by variance to calculate Mahalanobis Distance, then a chi2 p-value will be calculated based on mahalanobis distance. Then clusters with $ \geq 60\%$ samples with Bonferroni-corrected p-values < 0.05 were marked as outliers.

### D-statistic

For each sample, D-statistic represent the average correlation between its expression and other samples. A sample with lower D-statistic is more likely to be an outlier.



## Input

Gene expression in TPM. A data table with gene ID as first column and each sample ID as a subsquent columns.

## Output

1. A set of three diagnostic plots from each of the outlier detection method
2. A TPM file with low expression genes, samples with missing data, and outliers removed
3. Raw gene count file will also be processed to remove the same individuals and genes, as determined by the QC on TPM data.

## Minimal working example

The MWE data can be found on [Google Drive](https://drive.google.com/drive/u/0/folders/1Rv2bWHBbX_tastTh49ToYVDMV6rFP5Wk)

In [ ]:
sos run bulk_expression_QC.ipynb qc \
    --cwd output \
    --tpm-gct data/mwe.TPM.gct.gz \
    --counts-gct data/mwe.Counts.gct.gz \
    --container container/rna_quantification.sif

In [ ]:
head output/mwe.low_expression_filtered.log

```
[1] "Gene expression profiles loaded successfully!"
[1] "19 genes and 1118 samples are loaded from data/mwe.TPM.gct"
[1] "0 genes are filtered, because > 20 % samples have expression values < 0.1"
[1] "19 genes left, saving output."
```

In [ ]:
cat output/mwe.low_expression_filtered.outlier_removed.log

In [ ]:
ls output/*.pdf

```
output/mwe.low_expression_filtered.outlier_removed.tpm.gct.cluster.pdf  output/mwe.low_expression_filtered.outlier_removed.tpm.gct.D_stat_hist.pdf  output/mwe.low_expression_filtered.outlier_removed.tpm.gct.preQC_cluster.pdf  output/mwe.low_expression_filtered.outlier_removed.tpm.gct.RLEplot.pdf
```

## Command interface

In [1]:
sos run bulk_expression_QC.ipynb -h

usage: sos run bulk_expression_QC.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  qc

Global Workflow Options:
  --tpm-gct VAL (as path, required)
                        Required input is TPM file
  --counts-gct . (as path)
                        Raw counts file is optional and if available, it will be
                        filtered to match with the TPM file sample and genes
  --cwd output (as path)
  --container ''

Sections
  qc_1:
    Workflow Options:
      --low-expr-TPM 0.1 (as float)
      --low-expr-TPM-percent 0.2 (as float)
  qc_2:
    Workflow Options:
      --RLEFilterPercent 0.05 (as float)
      --DSFilterPercent 0.05 (as float)
      --topk-gen

## Workflow implementation

In [2]:
[global]
# Required input is TPM file
parameter: tpm_gct = path
# Raw counts file is optional and if available, it will be filtered to match with the TPM file sample and genes
parameter: counts_gct = path()
parameter: cwd = path("output")
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
cwd = path(f'{cwd:a}')
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"

# Number of threads
parameter: numThreads = 8

In [1]:
[qc_1 (basic check and low expression filtering)]
parameter: low_expr_TPM = 0.1
parameter: low_expr_TPM_percent = 0.2
input: tpm_gct
output: f'{cwd}/{_input:bnnn}.low_expression_filtered.tpm.gct.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand = "$[ ]", stderr = f'{_output:nnn}.stderr', stdout = f'{_output:nnn}.log',container = container, entrypoint = entrypoint
    # Load packages
    library(dplyr)
    library(readr)
    library(tibble)
    library(purrr)
    ## Setting inputs
    low_expr_TPM <- $[low_expr_TPM]
    low_expr_TPM_percent <- $[low_expr_TPM_percent]
    ## QC
    ### Data integrity check. The input file should has sample ID as column name; the first column is gene ID
    TPM_data <- read_tsv($[_input:ar], col_names = TRUE, comment = "#")%>%as.data.frame()
    names(TPM_data)[1] <- "feature"
    if( sum(duplicated(TPM_data$feature)) > 0){
      message("feature (e.g. gene names) should be in the first column. Please remove duplicated feature IDs, Exit!")
      quit(save = "no", status = 1, runLast = FALSE)
    }else{
      rownames(TPM_data) <- TPM_data$feature
      TPM_data = TPM_data[,-1]
      loaded_sample_count <- ncol(TPM_data)
    }
    #### NA check
    if(sum(is.na(TPM_data)) > 0 ){      
      message(paste0("NA is not allowed in the data, there are ",sum(is.na(TPM_data))," NAs, Exit!"))
      quit(save = "no", status = 1, runLast = FALSE)}
    #### make sure every expr column is in numeric type
    matrix_check <- map(TPM_data, is.numeric) %>% unlist
    if(sum(!matrix_check) > 0){
      message("The following column(s) in expression matrix is/are NOT in numeric type. Plese check, Proceed by excluding those samples")
      message(paste(names(matrix_check)[!matrix_check], collapse = "; "))
      TPM_data = TPM_data[,matrix_check]
    }
    message("Gene expression profiles loaded successfully!")
    message(paste(nrow(TPM_data), "genes and", ncol(TPM_data), "samples are loaded from", $[_input:r], sep = " "))
    
    #### Filter out low exp genes
    keep_genes_idx <- (rowMeans(TPM_data>low_expr_TPM)>low_expr_TPM_percent) 
    TPM_data = TPM_data[keep_genes_idx,]
    message(paste(sum(1 - keep_genes_idx), "genes are filtered by filter: >", low_expr_TPM_percent*100, "% samples have expression values <", low_expr_TPM))
    message(paste(sum(keep_genes_idx), "genes left, saving output to $[_output]."))
    TPM_data%>%as_tibble(rownames = "gene_ID")%>%write_delim("$[_output]","\t")

**CAUTION: I (GW) am not sure what to do with the offset for log-transformation on TPM. GTEx suggests using offset = 1. Here I keep the recommendation from [these authors](https://github.com/stormlovetao/eQTLQC/blob/master/Sample/src/report.Rmd) where both 0.0001 and 1 were used in different steps.**

In [ ]:
[qc_2 (remove outliers)]
parameter: RLEFilterPercent = 0.05
parameter: DSFilterPercent = 0.05
parameter: topk_genes = 100
parameter: cluster_percent = 0.6
parameter: pvalue_cutoff = 0.05
parameter: cluster_level = 5
output: f'{cwd}/{_input:bnnn}.outlier_removed.tpm.gct.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand = "$[ ]", stderr = f'{_output:nnn}.stderr', stdout = f'{_output:nnn}.log', container = container, entrypoint = entrypoint
    library(RColorBrewer)
    library(ape)
    library(reshape2)
    library(dplyr)
    library(readr)
    ## Setting parameters
    RLEFilterPercent <-$[RLEFilterPercent]
    DSFilterPercent <- $[DSFilterPercent]
    ### Hcluster parameter
    pvalues.cut <- $[pvalue_cutoff]
    topk_genes <- $[topk_genes]
    cluster_percent <- $[cluster_percent]
    treesNum <- $[cluster_level]
    ## Define functions to accomodate rank deficient covariance matrices in https://github.com/cumc/xqtl-pipeline/issues/307
    mahalanobis = function (x, center, cov, inverted = FALSE, ...) 
        {
            x <- if (is.vector(x)) 
                matrix(x, ncol = length(x))
            else as.matrix(x)
            if (!isFALSE(center)) 
                x <- sweep(x, 2L, center)
            if (!inverted) 
                cov <- MASS::ginv(cov, ...)
            setNames(rowSums(x %*% cov * x), rownames(x))
        }
          
    ## laod Data
    TPM_data <- read_tsv($[_input:ar], col_names = TRUE, comment = "#")%>%as.data.frame()
    rownames(TPM_data) <- TPM_data$gene_ID
    TPM_data = TPM_data[,-1]
    RLEFilterLength <- RLEFilterPercent*ncol(TPM_data)
    DSFilter <- DSFilterPercent*ncol(TPM_data)
  
    ## Outlier detection
  
    ### RLE
    # https://github.com/stormlovetao/eQTLQC/blob/86dcc388c8da7f1bd5b223f4b9b26f09c907eb15/Sample/src/report.Rmd#L71
    log_offset <- 0.0001
    logtpm = log10(TPM_data%>%as.matrix + log_offset)
    rle=logtpm-apply(logtpm, 1, median) # change "/" to "-" so that we got log(fold-change) which centered on 0 on the RLE plot.
    iqr = apply(rle,2,IQR)
    rle=melt( rle , variable.name = "Sample",value.name ="TPM", id="feature")
    names(rle) <- c("feature","Sample","TPM")
    rle_IQR <- rle %>% group_by(Sample) %>% summarise(IQR = IQR(TPM))
    rle_IQR_range <- rle_IQR$IQR %>% range %>% abs() %>% max()
    rle_IQR_range <- 2*rle_IQR_range %>% ceiling()
    bymedian <- with(rle, reorder(Sample, TPM, IQR))  # sort by IQR
    par(mar=c(3,3,3,3))
    pdf(file = "$[_output:n].RLEplot.pdf")
    boxplot(TPM ~ bymedian, data=rle, outline=F, ylim = c(-rle_IQR_range, rle_IQR_range), las=2, boxwex=1, col='gray', cex.axis=0.3, main="RLE plot before QC", xlab="", ylab="Residual expression levels", frame=F)
    dev.off()
    ExpPerSample <- nrow(TPM_data)
    RLEFilterList <- unique(levels(bymedian)[((length(levels(bymedian))-(RLEFilterLength))+1):(length(levels(bymedian))+1)])
    RLEFilterList <- as.character(RLEFilterList)
    message(paste0("The right most ", RLEFilterPercent*100, "% samples (N = ", length(RLEFilterList), ") are marked as candidate outliers in this step:") )
    for (x in 1:length(RLEFilterList)){
      message(RLEFilterList[x])
    }

    ### hcluster  
    sampleDists <- 1 - cor(logtpm, method='spearman')
    hc <- hclust(as.dist(sampleDists), method = "complete")
    hcphy <- as.phylo(hc)
  
    pdf(file = "$[_output:n].preQC_cluster.pdf")
    plot(hcphy, type = "unrooted", cex=.2, lab4ut='axial',underscore = T, main="Sample clustering before QC (Spearman - Cor.)")
    dev.off()
    # https://github.com/stormlovetao/eQTLQC/blob/86dcc388c8da7f1bd5b223f4b9b26f09c907eb15/Sample/src/report.Rmd#L102
    log_offset <- 1
    logtpm = log10(TPM_data%>%as.matrix + log_offset)
    ntop <- topk_genes
    Pvars <- apply(logtpm, 1, var)
    select <- order(Pvars, decreasing =TRUE)[seq_len(min(ntop, length(Pvars)))]
    MD_matrix <- logtpm[select, ]
    MahalanobisDistance = mahalanobis(t(MD_matrix), colMeans(t(MD_matrix)), cov(t(MD_matrix))) 
    # Note: t(MD_matrix)=sample_row*gene_column, Manhalanobis() returns one vector with length=row number
    pvalues = pchisq(MahalanobisDistance, df=nrow(MD_matrix), lower.tail=F)
    pvalues.adjust = p.adjust(pvalues, method ="bonferroni") # adjusted pvalues for each sample
    pvalues.low <- pvalues.adjust[pvalues.adjust<pvalues.cut]
    
    HCoutliers <- character()
    for(x in c(1:treesNum)){
      trees <- cutree(hc,k=x)
      idx <- c(1:x)#which tree is checking
      for(i in idx)
      {
        group <- hc$labels[which(trees == i)]
        if(sum(group %in% names(pvalues.low))/length(group) >= cluster_percent)
        {
          HCoutliers <- union(HCoutliers,group)
        }
      }
    }
    
    message(paste(length(HCoutliers), "samples are marked as candidate outlier(s) in this step.", sep = " "))
    if(length(HCoutliers)>0){
      message("Sample outliers are marked in red as follows:")
      for (x in 1:length(HCoutliers)){
          message(HCoutliers[x])
      }
      co1 = hc$labels%in%HCoutliers
      co1[which(co1 == "FALSE")]<-"gray0"
      co1[which(co1 == "TRUE")]<-"red"
      par(mar=c(3,3,3,3))
  
    pdf(file = "$[_output:n].cluster.pdf")
    plot(hcphy, tip.col = co1, type = "unrooted", cex=.2, lab4ut='axial',underscore = T, main="Label Outliers in Red")
    Xcol = c("gray0", "red")
    Xtext = c("Normal Sample", "Outliers")
    legend('bottomleft',pch=21,Xtext, col='white',pt.bg=Xcol, cex=1)
    dev.off()
    }else{
      message("No outlier detected.")
    }
    
    ### D-s
    D = apply(1-sampleDists, 1, median)
    pdf(file = "$[_output:n].D_stat_hist.pdf")
    hist(D, breaks=100, ylab="Number of samples", xlab="D-statistic", main="Histogram of Sample D-statistics before data QC")
    dev.off()
    
    DSFilter <- sort(D)[DSFilter]
    D<-as.data.frame(D)
    D<-data.frame(Sample = rownames(D),D = D$D)
    D_filterList = D%>%filter(D <= DSFilter)
    D_filterList <- D_filterList$Sample
    D_filterList<-as.character(D_filterList)
    message(paste0("The right most ", DSFilterPercent*100, "% samples (N=", length(D_filterList), ") are marked as candidate outliers in this step:") )
    for (x in 1:length(D_filterList)){
      message(D_filterList[x])
    }
    
    ## Outliers are the intersect of three candidates list
    outliersList <- intersect(RLEFilterList,intersect(HCoutliers,D_filterList))
    message("Outliers:")
    for (x in 1:length(outliersList)){
      message(outliersList[x])
    }
    outliersIndex <- which(colnames(logtpm) %in% outliersList)
    if(!length(outliersIndex) == 0){
        TPM_data <- TPM_data[,-outliersIndex]
    }
    ## Add 2 header lines, https://github.com/getzlab/rnaseqc/blob/286f99dfd4164d33014241dd4f3149da0cddf5bf/src/RNASeQC.cpp#L426
    cat(paste("#1.2\n#", nrow(TPM_data), ncol(TPM_data), "\n"), file=$[_output:nr], append=FALSE)
    TPM_data%>%as_tibble(rownames = "gene_ID")%>%write_delim($[_output:nr],delim = "\t",col_names = T, append = T)
  
bash: expand = "$[ ]", stderr = f'{_output:nnn}.stderr', stdout = f'{_output:nnn}.log', container = container, entrypoint = entrypoint
    gzip -f --best $[_output:n]

Additional formatting:

1. Filter out the geneCount table based on TPM table.
2. Adds two comment lines above the header of TPM and geneCount table to mimick the original output from RNASeQC.

In [ ]:
[qc_3 (remove gene and samples from raw counts)]
stop_if(not counts_gct.is_file())
output: f'{cwd}/{_input:bnnn}.geneCount.gct.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads
R: expand = "$[ ]", stderr = f'{_output:nn}.stderr', stdout = f'{_output:nn}.stdout', container = container, entrypoint = entrypoint
    library("dplyr")
    library("readr")
    # Reason to use read.table: 1.accomodate both " " and "\t"
    tpm = read_delim($[_input:ar], "\t", col_names = T, comment = "#")
    geneCount = read_delim($[counts_gct:ar], "\t" ,col_names = T, comment = "#")
    ## Make geneCount consistant with tpm, remove gene by filter() and remove samples by select()
    geneCount = geneCount%>%filter(gene_ID %in% tpm$gene_ID)%>%select(colnames(tpm))
    ## Add 2 header lines, https://github.com/getzlab/rnaseqc/blob/286f99dfd4164d33014241dd4f3149da0cddf5bf/src/RNASeQC.cpp#L426
    cat(paste("#1.2\n#", nrow(geneCount), ncol(geneCount) - 1, "\n"), file=$[_output:nr], append=FALSE)
    ## Save each file with 3 header line
    geneCount%>%write_delim($[_output:nr],delim = "\t",col_names = T, append = T)

bash: expand = "$[ ]", stderr = f'{_output:nnn}.stderr', stdout = f'{_output:nnn}.log', container = container, entrypoint = entrypoint
    gzip -f --best $[_output:n]